# Interval Algebra Examples

<i>Version 1</i>

## References

1. ["Maintaining Knowledge about Temporal Intervals" by James F. Allen](https://cse.unl.edu/~choueiry/Documents/Allen-CACM1983.pdf) - Allen's original paper (PDF)
1. [Allen's Interval Algebra](https://www.ics.uci.edu/~alspaugh/cls/shr/allen.html) or [here](https://thomasalspaugh.org/pub/fnd/allen.html) - summarizes Allen's algebra of proper time intervals
1. ["Intervals, Points, and Branching Time" by A.J. Reich](https://www.researchgate.net/publication/220810644_Intervals_Points_and_Branching_Time) - basis for the extensions here to Allen's algebra
1. [W3C Time Ontology in OWL](https://www.w3.org/TR/owl-time/) - temporal vocabulary used here is based on the W3C vocabulary of time
1. [bitsets Python package](https://bitsets.readthedocs.io/en/stable/) - used to implement Algebra relation sets and operations
1. [NetworkX Python package](http://networkx.github.io/) - used to represent directed graph of constraints
1. [Python format string syntax](https://docs.python.org/3/library/string.html#format-string-syntax) - used in Algebra summary method
1. [Spatial Ontology](https://www.w3.org/2017/sdwig/bp/) - I'm still looking for a standard spatial vocabulary; maybe start here
1. [Qualitative Spatial Relations (QSR) Library](https://qsrlib.readthedocs.io/en/latest/index.html) - an alternative library to the one defined here

## Dependencies

In [1]:
import qualreas as qr
import os

In [2]:
path = os.path.join(os.getenv('PYPROJ'), 'qualreas')

## Instantiate an Algebra

In [18]:
#alg = qr.Algebra(os.path.join(path, "Algebras/LinearIntervalAlgebra.json"))  # Allen's algebra of proper time intervals
alg = qr.Algebra(os.path.join(path, "Algebras/ExtendedLinearIntervalAlgebra.json"))
#alg = Algebra(os.path.join(path, "Algebras/LeftBranchingIntervalAlgebra.json"))
#alg = Algebra(os.path.join(path, "Algebras/RightBranchingIntervalAlgebra.json"))
#alg = Algebra(os.path.join(path, "Algebras/RCC8Algebra.json"))

### Algebra Summary

In [19]:
alg.summary()

  Algebra Name: ExtendedLinearIntervalAlgebra
   Description: Extension of Allen's algebra to include Instants/Points (P)
 Equality Rels: E|PE
     Relations:
            NAME (ABBREV)         CONVERSE (ABBREV)  REFLEXIVE  SYMMETRIC TRANSITIVE   DOMAIN        RANGE
             Before (  B)               After ( BI)    False      False       True    Pt|PInt       Pt|PInt
              After ( BI)              Before (  B)    False      False       True    Pt|PInt       Pt|PInt
             During (  D)            Contains ( DI)    False      False       True    Pt|PInt          PInt
           Contains ( DI)              During (  D)    False      False       True       PInt       Pt|PInt
             Equals (  E)              Equals (  E)     True       True       True       PInt          PInt
           Finishes (  F)         Finished-by ( FI)    False      False       True       PInt          PInt
        Finished-by ( FI)            Finishes (  F)    False      False       True    

### Algebra Element Summary

In [20]:
alg.element_summary('B')

                    Name: Before
                  Domain: ['Point', 'ProperInterval']
                   Range: ['Point', 'ProperInterval']
                Converse: After
           Is Reflexive?: False
           Is Symmetric?: False
          Is Transitive?: True
Is an Equality Relation?: False


In [22]:
alg.element_summary('PS')

                    Name: Point-Starts
                  Domain: ['Point']
                   Range: ['ProperInterval']
                Converse: Point-Started-By
           Is Reflexive?: False
           Is Symmetric?: False
          Is Transitive?: False
Is an Equality Relation?: False


#### Equality Relations

#### The number and type of equality relations in an algebra depends on the number and type of domains and ranges supported by the algebra

In [34]:
alg.all_equality_relations

relset(['E', 'PE'])

In [35]:
alg.element_summary('E')

                    Name: Equals
                  Domain: ['ProperInterval']
                   Range: ['ProperInterval']
                Converse: Equals
           Is Reflexive?: True
           Is Symmetric?: True
          Is Transitive?: True
Is an Equality Relation?: True


In [36]:
alg.element_summary('PE')

                    Name: Point-Equals
                  Domain: ['Point']
                   Range: ['Point']
                Converse: Point-Equals
           Is Reflexive?: True
           Is Symmetric?: True
          Is Transitive?: True
Is an Equality Relation?: True


### Creating Relation Sets

#### There are two acceptable input formats

In [37]:
relset_version1 = alg.relset("B|M|FI")
relset_version2 = alg.relset(['B', 'FI', 'M'])
print(relset_version1)
print(relset_version2)
print(f"Same? {relset_version1 == relset_version2}")

B|FI|M
B|FI|M
Same? True


#### Singleton sets can also be input in two acceptable ways

In [38]:
singleton_relset_v1 = alg.relset("B")
singleton_relset_v2 = alg.relset(["B"])
print(singleton_relset_v1)
print(singleton_relset_v2)
print(f"Same? {singleton_relset_v1 == singleton_relset_v2}")

B
B
Same? True


#### Not really sure why one would want to create the empty set, but you can, in two different ways

In [39]:
empty_relset_v1 = alg.relset("")
empty_relset_v2 = alg.relset([])
print(empty_relset_v1)
print(empty_relset_v2)
print(f"Same? {empty_relset_v1 == empty_relset_v2}")



Same? True


### Operations on Relation Sets

#### Addition (+) is set intersection:

In [40]:
alg.relset('B|M|O') + alg.relset('F|O|M|S')

relset(['M', 'O'])

In [41]:
alg.relset('B|M|O') + alg.relset('F|S')

relset()

#### "Multiplication" is relation composition applied to sets of relations

In [42]:
alg.compose(alg.relset("F"), alg.relset("O"))

relset(['D', 'O', 'S'])

In [43]:
alg.compose(alg.relset("MI"), alg.relset("D"))

relset(['D', 'F', 'OI'])

When more that one relation appears in the sets, the result of composition is the union of all pairwise compositions of the individual relations in the sets.

In [44]:
alg.compose(alg.relset("F|MI"), alg.relset("O|D"))

relset(['D', 'F', 'O', 'OI', 'S'])

#### Converses

##### Individual relations have converses

In [29]:
rel_symbol = 'B'
print(f"The converse of {alg.rel_name(rel_symbol)} is {alg.rel_converse_name(rel_symbol)}")

The converse of Before is After


##### And relation sets have converses

In [30]:
before = alg.relset('B')
after = alg.converse(alg.relset('B'))
print(f"{before} is the converse of {after}.")
print(f"{alg.converse(relset_version1)} is the converse of {relset_version1}")

B is the converse of BI.
BI|F|MI is the converse of B|FI|M
